# Script de pré-processamento para as bases de dados do ENEM

**Autor**: Rafael Victor Araujo Bernardes - rafaelvictor.bernardes@gmail.com

## Introdução

O seguinte _script_ tem como propósito realizar a preparação dos dados para a subsequente aplicação dos métodos de clusterização e seleção de atributos propostos em meu Trabalho de Conclusão de Curso (TCC). Para tanto, o _script_ realizará sequencialmente o processamento e as transformações necessárias dos dados. O processamento dos dados é uma etapa que tem como objetivo geral: limpar, corrigir ou remover dados inconsistentes, tratar dados ausentes e identificar anomalias (_outliers_). Já o processo de transformação dos dados visa realizar a normalização, agregação, criação de novos atributos, redução e sintetização dos dados, entre outros. Estas etapas correspondem, respectivamente, à segunda e à terceira etapa do método _Knowledge Discovery in Databases_ (KDD) de Fayyad et al. (1996).

A princípio, o código aqui desenvolvido será aplicado nas bases de dados do ENEM referentes aos anos de 2022, 2020 e 2019. A escolha desses anos deve-se à alta correção dos atributos presentes nessas bases. Além disso, cada um desses anos está correlacionado a algum evento histórico cujo impacto na distribuição dos dados deseja-se investigar.

Espera-se que ao final da execução deste script, todas as bases alvo sejam transformada em bases menores, corrigidas, normalizadas e preparadas para aplicação de outras tecnicas.

## Importação dos dados e recursos necessários

As bases de dados utilizadas para o desenvolvimento deste trabalho (microdados) podem ser encontradas no portal do Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira (INEP), o órgão responsável pelo ENEM, através do link: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem.

Cada microdado, por seu turno, contêm uma variedade de informações sobre os participantes coletadas ao longo de todo o processo do exame. Ao realizar o download, os usuários encontrarão, não apenas a própria base de dados, como também: as provas, gabaritos, informações sobre questões, notas, questionários respondidos pelos inscritos, documentos técnicos e, acima de tudo, um extenso dicionário relacionado ao conjunto de dados. Este dicionário se mostra especialmente relevante para o desenvolvimento deste trabalho uma vez que ele caracteriza objetivamente todas as colunas presentes na base de dados.

Vale ressaltar que os microdados estão formatados em arquivos de extesão ".csv" e, para os anos selecionados, cada tabela contêm cerca de 76 colunas. Devido à grande quantidade de colunas e à diversidade de possíveis respostas que cada coluna pode conter, torna-se inviável realizar uma caracterização completa da base neste trabalho. No entanto, todas as colunas relevantes para o desenvolvimento desta pesquisa terão seus significados explorados em momentos oportunos. Recomenda-se a leitura dos dicionários de dados referêntes aos anos propostos.

In [1]:
import pandas as pd

In [2]:
# Variáveis de controle

ano = '2019'
# ano = '2022'
# ano = '2020'

# Endereço dados brutos:
DATASET_ENEM_PATH = 'D:\Bases\MICRODADOS_ENEM_' + ano + '.csv'

# O novo arquivo será salvo neste endereço:
PATH_DATASET_TRATADO = 'D:\\BASES_PRE_PROCESSADAS\\PRE_PROCESSADOS_ENEM_' + ano + '.csv'

In [3]:
microdadosEnem = pd.read_csv(DATASET_ENEM_PATH, sep=';', encoding='ISO-8859-1')

## Análise exploratória dos dados.

O objetivo principal desta análise exploratória dos dados é entender e extrair _insights_ iniciais sobre os microdados por meio da observação de padrões, tendências, relações e anomalias. Segue abaixo um compilado das etapas que segui:

In [4]:
# Contato inicial com a base

microdadosEnem.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001595656,2019,13,M,1,3,1,1,11,1,...,A,A,A,A,A,A,C,A,C,B
1,190001421546,2019,8,M,1,1,1,1,5,1,...,A,A,A,B,A,A,E,A,A,B
2,190001133210,2019,13,F,1,3,1,1,13,1,...,A,A,A,B,A,A,C,A,A,B
3,190001199383,2019,10,F,1,1,1,1,4,1,...,A,A,A,C,A,B,D,A,B,B
4,190001237802,2019,7,F,1,1,1,1,4,1,...,B,A,A,B,A,A,C,B,B,B


In [5]:
# Visualizando as dimensões da base

microdadosEnem.shape

(5095171, 76)

In [6]:
# Visualizando as colunas da base

microdadosEnem.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE',
       'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
       'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC',
       'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA',
       'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA',
       'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', '

## Pré-processamento dos dados

O objetivo principal deste pré-processamento dos dados é preparar e otimizar os dados brutos para análise ou modelagem, visando melhorar a qualidade, a eficácia e a eficiência das etapas subsequentes.

### Eliminando características individuais, de baixa variância ou inexpressivas.

Por meio do dicionário de dados, sabe-se que as colunas abaixo possuem pouca relevância para o objetivo da análise. Isto porque algumas delas possuem variância igual a zero (como é o caso da coluna "NU_ANO" que estará sempre preenchida com o ano de realização da prova) e outras representam caracteristicas altamente individuais dos candidatos e, portanto, não são aptas para avaliação de nenhum tipo de tendência (como é o caso das colunas com prefixo "TX_" que são os vetores com as respostas objetivas para as diferentes áreas de conhecimento da prova de cada candidato). Há ainda um conjunto de colunas inexpressivas que tratam sobre a cor de prova do participante. Todas estas mencionadas serão removidas.

* "NU_INSCRICAO" - Número de inscrição (Individual),
* "NU_ANO" - Ano do Enem (Variância Zero),
* "TX_RESPOSTAS_CN" - Vetor com as respostas da parte objetiva da prova de Ciências da Natureza (Individual),
* "TX_RESPOSTAS_CH" - Vetor com as respostas da parte objetiva da prova de Ciências Humanas (Individual),
* "TX_RESPOSTAS_LC" - Vetor com as respostas da parte objetiva da prova de Linguagens e Códigos (Individual),
* "TX_RESPOSTAS_MT" - Vetor com as respostas da parte objetiva da prova de Matemática (Individual),
* "TX_GABARITO_CN" - Vetor com o gabarito da parte objetiva da prova de Ciências da Natureza (Individual),
* "TX_GABARITO_CH" - Vetor com o gabarito da parte objetiva da prova de Ciências Humanas (Individual),
* "TX_GABARITO_LC" - Vetor com o gabarito da parte objetiva da prova de Linguagens e Códigos (Individual),
* "TX_GABARITO_MT" - Vetor com o gabarito da parte objetiva da prova de Matemática (Individual),
* "CO_PROVA_CN" - Código do tipo de prova de Ciências da Natureza (Inexpressivo),
* "CO_PROVA_CH" - Código do tipo de prova de Ciências Humanas (Inexpressivo),
* "CO_PROVA_LC" - Código do tipo de prova de Linguagens e Códigos (Inexpressivo),
* "CO_PROVA_MT" - Código do tipo de prova de Matemática (Inexpressivo)


In [7]:
microdadosEnem.drop(columns=[
    'NU_INSCRICAO',
    # 'NU_ANO', - Vai ser removido um pouco mais adiante
    'TX_RESPOSTAS_CN',
    'TX_RESPOSTAS_CH',
    'TX_RESPOSTAS_LC',
    'TX_RESPOSTAS_MT',
    'TX_GABARITO_CN',
    'TX_GABARITO_CH',
    'TX_GABARITO_LC',
    'TX_GABARITO_MT',
    'CO_PROVA_CN',
    'CO_PROVA_CH',
    'CO_PROVA_LC',
    'CO_PROVA_MT'
], inplace=True)

microdadosEnem.head()

,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,2019,13,M,1,3,1,1,11,1,NaN,...,A,A,A,A,A,A,C,A,C,B
1,2019,8,M,1,1,1,1,5,1,1.0,...,A,A,A,B,A,A,E,A,A,B
2,2019,13,F,1,3,1,1,13,1,1.0,...,A,A,A,B,A,A,C,A,A,B
3,2019,10,F,1,1,1,1,4,1,NaN,...,A,A,A,C,A,B,D,A,B,B
4,2019,7,F,1,1,1,1,4,1,1.0,...,B,A,A,B,A,A,C,B,B,B


### Eliminando redundâncias

Além disso, irei eliminar colunas redundantes.

Exemplo: "NO_MUNICIPIO_RESIDENCIA" (nome do município) equivalente a "CO_MUNICIPIO_RESIDENCIA" (código do município).

In [8]:
microdadosEnem.drop(columns=[
    'NO_MUNICIPIO_ESC','CO_UF_ESC','SG_UF_ESC', # Todas estas colunas são iguais à CO_MUNICIPIO_ESC
    'NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA', # Todas estas colunas são iguais à CO_MUNICIPIO_PROVA
    'TP_ANO_CONCLUIU' # Ano de Conclusão do Ensino Médio - Essa informação pode ser mais facilmente extraída da coluna TP_ST_CONCLUSAO
], inplace=True)

microdadosEnem.head()

,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,2019,13,M,1,3,1,1,1,NaN,0,...,A,A,A,A,A,A,C,A,C,B
1,2019,8,M,1,1,1,1,1,1.0,0,...,A,A,A,B,A,A,E,A,A,B
2,2019,13,F,1,3,1,1,1,1.0,0,...,A,A,A,B,A,A,C,A,A,B
3,2019,10,F,1,1,1,1,1,NaN,0,...,A,A,A,C,A,B,D,A,B,B
4,2019,7,F,1,1,1,1,1,1.0,0,...,B,A,A,B,A,A,C,B,B,B


### Tratamento de dados faltantes

In [9]:
# Verificando se existem valores nulos

microdadosEnem.isnull().values.any()

True

True == A base contém dados nulos;
False == A base não contém dados nulos 

Para lidar com os dados ausentes no _dataset_, uma possível abordagem seria remover todas as linhas que contenham quaisquer valores nulos. Ao final deste processo, haveria um conjunto de dados compostos apenas por linhas que possuem todas as colunas preenchidas.

Apesar da simplicidade de implementação desta técnica, esta abordagem se mostrou inviável devido a alta perda de informações. Ao realizar um estudo de caso nas bases propostas, pude perceber que a implementação desta técnica ocarionaria na perda média de 70% dos registros presentes nas bases de dados.

Desse modo, será necessário adotar estratégias específicas para tratar cada coluna que contém valores nulos.
Primeiro é necessário compreender onde estão os valores nulos:

In [10]:
# Verificando onde estão os valores nulos

microdadosEnem.isnull().sum()

NU_ANO                          0
TP_FAIXA_ETARIA                 0
TP_SEXO                         0
TP_ESTADO_CIVIL                 0
TP_COR_RACA                     0
TP_NACIONALIDADE                0
TP_ST_CONCLUSAO                 0
TP_ESCOLA                       0
TP_ENSINO                 2217036
IN_TREINEIRO                    0
CO_MUNICIPIO_ESC          3947784
TP_DEPENDENCIA_ADM_ESC    3947784
TP_LOCALIZACAO_ESC        3947784
TP_SIT_FUNC_ESC           3947784
CO_MUNICIPIO_PROVA              0
TP_PRESENCA_CN                  0
TP_PRESENCA_CH                  0
TP_PRESENCA_LC                  0
TP_PRESENCA_MT                  0
NU_NOTA_CN                1384836
NU_NOTA_CH                1172125
NU_NOTA_LC                1172125
NU_NOTA_MT                1384836
TP_LINGUA                       0
TP_STATUS_REDACAO         1172126
NU_NOTA_COMP1             1172126
NU_NOTA_COMP2             1172126
NU_NOTA_COMP3             1172126
NU_NOTA_COMP4             1172126
NU_NOTA_COMP5 

##### ESTRATÉGIAS:

1. É possivel observar que há uma quantidade significativa de registros com informações ausentes relacionadas à **escola do candidato**. São elas:

    * "CO_MUNICIPIO_ESC" (Código do município da escola);
    * "TP_DEPENDENCIA_ADM_ESC" (Dependência administrativa da escola);
    * "TP_LOCALIZACAO_ESC" (Tipo de localização da escola); 
    * "TP_SIT_FUNC_ESC" (Situação de funcionamento da escola);
    * "TP_ENSINO" (Tipo de instituição de ensino);

Acredita-se que estas informações relacionadas a escola do candidato não eram de preenchimento obrigatório no momento cadastro do participante no exame. Isto justificaria a grande ausência de informações.

Dado que aproximadamente 70% das linhas da tabela total não possuem esses valores preenchidos, vou optar por remover essas colunas do modelo, pois não será possivel utiliza-las como objeto de ánalise confiável e nem fazer nenhum outro tipo de inferência preditiva ou classificativa.

Sendo assim, nenhum tipo de inferência ou analise sobre a escola do candidato será feita, mas conseguiremos preservar 70% a mais de dados.

_Uma sugestão de trabalho para o futuro é fazer analises relacionadas justamente as escolas dos participantes._

In [11]:
# Removendo as colunas não confiáveis

microdadosEnem.drop(columns=[
    'CO_MUNICIPIO_ESC',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC',
    'TP_ENSINO'
], inplace=True)

2. As colunas relacionadas as notas dos participantes também demonstram ter uma quantidade grande de dados nulos. 

Acredita-se que estas colunas possuam registros vazios devido a ausência do candidato no dia de aplicação do exame. Desta forma, **vou optar por prosseguir apenas com os participantes presentes nos dois dias de aplicação da prova**.

Ao optar por isto, é fato que estarei reduzindo minha quantidade de registros, mas poderei também descartar as colunas relacionadas a presença dos candidatos e simplificar meu modelo.

(Estas colunas deverão ter variância zero. Ou seja, apenas candidatos presentes)

_Uma sugestão de trabalho para o futuro é fazer a analise justamente destas linhas que estou desconsiderando._

In [12]:
# Removendo participantes ausêntes por meio da análise das notas

microdadosEnem.dropna(subset=['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'], inplace=True)

In [13]:
# Validando se realmente posso remover as colunas de presença por meio da análise de variância.

colunasDePresença = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']
microdadosEnem[colunasDePresença].var()

TP_PRESENCA_CN    0.0
TP_PRESENCA_CH    0.0
TP_PRESENCA_LC    0.0
TP_PRESENCA_MT    0.0
dtype: float64

In [14]:
# Dado que a hipotese foi validada, posso remover as colunas de presença para simplificar meu modelo

microdadosEnem.drop(columns=colunasDePresença, inplace=True)

Por fim, serão descartados os candidatos que possuirem alguma informação não preenchida no questionário socioeconômico principal.

**OBS**: Este procedimento está excluindo poucos dados do _dataset_. Isso porque não é comum que o questionário socioeconômico principal possua dados ausentes.

In [15]:
microdadosEnem.dropna(subset=[
    'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
    'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
    'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
    'Q023', 'Q024', 'Q025'
], inplace=True)

##### REAVALIAÇÃO DA CONSISTÊNCIA DOS DADOS:

Se tudo tiver ocorrido conforme o planejado, não devem ter mais dados nulos na tabela

In [16]:
microdadosEnem.isnull().sum()

NU_ANO                0
TP_FAIXA_ETARIA       0
TP_SEXO               0
TP_ESTADO_CIVIL       0
TP_COR_RACA           0
TP_NACIONALIDADE      0
TP_ST_CONCLUSAO       0
TP_ESCOLA             0
IN_TREINEIRO          0
CO_MUNICIPIO_PROVA    0
NU_NOTA_CN            0
NU_NOTA_CH            0
NU_NOTA_LC            0
NU_NOTA_MT            0
TP_LINGUA             0
TP_STATUS_REDACAO     0
NU_NOTA_COMP1         0
NU_NOTA_COMP2         0
NU_NOTA_COMP3         0
NU_NOTA_COMP4         0
NU_NOTA_COMP5         0
NU_NOTA_REDACAO       0
Q001                  0
Q002                  0
Q003                  0
Q004                  0
Q005                  0
Q006                  0
Q007                  0
Q008                  0
Q009                  0
Q010                  0
Q011                  0
Q012                  0
Q013                  0
Q014                  0
Q015                  0
Q016                  0
Q017                  0
Q018                  0
Q019                  0
Q020            

## Transformação dos dados

Esta pesquisa visa realizar uma análise histórica das bases de dados do ENEM e identificar tendências temporais nas características de candidatos quando dividios em _clusters_.

Em outras palavras, queremos entender se as caracteristicas relacionadas a _clusters_ de candidatos com predominância de notas baixas e altas se alterou ao longo do tempo.

Dessa forma, é necessário adotar uma forma simples para mensurar se a nota do partipante foi alta ou baixa. Isso será de grande importância para realização das analises quando os candidados estiverem clusterizados.

A estratégia adotada será a de criação de uma nova coluna que contenha a média simples de todas as notas que o candidato obteve.

In [17]:
microdadosEnem['MEDIA_NOTAS'] = (
    microdadosEnem.NU_NOTA_CN +
    microdadosEnem.NU_NOTA_CH +
    microdadosEnem.NU_NOTA_LC +
    microdadosEnem.NU_NOTA_MT +
    microdadosEnem.NU_NOTA_REDACAO
)/5

In [18]:
microdadosEnem.head()

,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,IN_TREINEIRO,CO_MUNICIPIO_PROVA,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,MEDIA_NOTAS
3,2019,10,F,1,1,1,1,1,0,1721000,...,A,A,C,A,B,D,A,B,B,475.34
4,2019,7,F,1,1,1,1,1,0,3118601,...,A,A,B,A,A,C,B,B,B,639.36
5,2019,13,M,2,2,1,1,1,0,5107602,...,A,A,B,A,A,E,A,B,B,649.62
6,2019,7,F,1,3,1,1,1,0,2924009,...,A,A,B,A,A,B,A,A,A,534.98
9,2019,2,M,1,1,1,2,2,0,3550308,...,A,A,B,A,B,C,B,B,B,495.04


Agora, seguirei com o descarte das colunas referênte as notas dos candidatos nas áreas do conhecimento, dado ques estas colunas já estão representadas na coluna de média.

Além disso, removerei as colunas relacionadas as competências e ao status da redação, dado que estas colunas nada mais são que componentes já representados pela coluna de média.

Caso deixassemos estas colunas presentes no dataset, elas sempre seriam evidenciadas como os fatores mais relevantes para progressão das notas (obviamente).

_EX: Não possuir a redação com status de anulada sempre significará maior média de nota._ 

In [19]:
microdadosEnem.drop(columns=[
    'NU_NOTA_CN',
    'NU_NOTA_CH',
    'NU_NOTA_LC',
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
    'NU_NOTA_COMP1',
    'NU_NOTA_COMP2',
    'NU_NOTA_COMP3',
    'NU_NOTA_COMP4',
    'NU_NOTA_COMP5',
    'TP_STATUS_REDACAO'
], inplace=True)

In [20]:
microdadosEnem.head()

,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,IN_TREINEIRO,CO_MUNICIPIO_PROVA,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,MEDIA_NOTAS
3,2019,10,F,1,1,1,1,1,0,1721000,...,A,A,C,A,B,D,A,B,B,475.34
4,2019,7,F,1,1,1,1,1,0,3118601,...,A,A,B,A,A,C,B,B,B,639.36
5,2019,13,M,2,2,1,1,1,0,5107602,...,A,A,B,A,A,E,A,B,B,649.62
6,2019,7,F,1,3,1,1,1,0,2924009,...,A,A,B,A,A,B,A,A,A,534.98
9,2019,2,M,1,1,1,2,2,0,3550308,...,A,A,B,A,B,C,B,B,B,495.04


### Ajustes especificos para cada ano

#### Ajustes relacionados ao ano de 2022


1) Em 2022 a coluna "TP_COR_RACA" possuía uma opção a mais chamada "Não dispõe da informação" (Item: 6). 

Esta opção não existia nos anos anteriores. Acredito que seja possível normalizar os dados de 2022 para ficar mais coerente com os anos anteriores

**IDEA**: Unificar com a resposta "Não declarado" (Item: 0)

2) O ano de 2022 deixou te ter a resposta "Exterior" (item: 4) para a coluna "TP_ESCOLA"

**IDEA**: Para resolver esta questão, acredito que o melhor caminho seja remover de todas as demais tabelas os alunos que preencheram a opção "Exterior".

In [21]:
primeiro_item = microdadosEnem.iloc[0]  # Obtendo o primeiro item do dataset

if primeiro_item['NU_ANO'] == 2022:
    microdadosEnem['TP_COR_RACA'].replace(6, 0, inplace=True)
    print("Itens 6 substituídos por 0 na coluna TP_COR_RACA.")
else:
    microdadosEnem = microdadosEnem[microdadosEnem['TP_ESCOLA'] != 4]
    print("Exterior removido como opção válida para a coluna - Tipo de escola do Ensino Médio")

microdadosEnem.head()
microdadosEnem.drop(columns=['NU_ANO'], inplace=True)

Exterior removido como opção válida para a coluna - Tipo de escola do Ensino Médio


### Tratando variáveis categóricas

O tratamento adequado de variáveis categóricas desempenha um papel crucial na análise e modelagem de dados. As variáveis categóricas representam características qualitativas que não podem ser diretamente utilizadas em algoritmos de aprendizado de máquina. Portanto, é essencial convertê-las em uma forma numérica que os modelos possam compreender.

In [22]:
microdadosEnem.select_dtypes(include='object').describe()

,TP_SEXO,Q001,Q002,Q003,Q004,Q006,Q007,Q008,Q009,Q010,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
count,3701909,3701909,3701909,3701909,3701909,3701909,3701909,3701909,3701909,3701909,...,3701909,3701909,3701909,3701909,3701909,3701909,3701909,3701909,3701909,3701909
unique,2,8,8,6,6,17,4,5,5,5,...,5,5,2,5,2,2,5,2,5,2
top,F,E,E,C,B,C,A,B,C,A,...,B,A,A,B,A,A,C,A,B,B
freq,2201154,1006235,1234749,864812,1544590,904985,3409483,2516690,1845910,2000030,...,1854291,3603350,2932196,2491219,2526264,2833238,1110546,2769769,1636744,2919960


#### _get_dummies_ VS _OneHotEncoder_

O OneHotEncoder e o get_dummies são técnicas comuns para lidar com variáveis categóricas em análises de dados e modelagem estatística. Ambos convertem variáveis categóricas em formato numérico para que possam ser utilizadas em algoritmos de aprendizado de máquina. A principal semelhança entre eles é que ambos criam variáveis indicadoras para cada categoria da variável categórica, transformando-as em variáveis binárias (0s e 1s).

Por exemplo: se existir uma coluna chamada "Cor" com categorias "Vermelho", "Verde" e "Azul". Ambas as funções poderiam criar três novas colunas: "Cor_Vermelho", "Cor_Verde" e "Cor_Azul". Cada linha terá um valor 1 na coluna correspondente à cor daquela linha e 0 nas outras colunas. Isso permite que as informações categóricas sejam tratadas como variáveis numéricas, facilitando a análise estatística e o uso em modelos de machine learning (ML).

Ainda que os métodos possuam grandes semelhanças, seus casos de uso são um pouco distintos. O OneHotEncoder faz parte da biblioteca Scikit-Learn e é frequentemente aplicado como parte de um pipeline de pré-processamento de dados. Este método é especialmente utilizado quando o desenvolvimento está relacionado ao aprendizado de máquina. Isso porque o OneHotEncoder tem uma gestão melhor de variáveis não mapeadas - evitando erros - e o processamento que foi criado pelo OneHotEncoder pode ser armazenado em arquivos pickle, permitindo o uso em outros projetos.

Por outro lado, o get_dummies, que faz parte da biblioteca pandas em Python, é mais simples e direto. Sua única responsabilidade é criar colunas separadas para cada categoria da variável categórica. Descarta-se a complexidade adicional de integração com pipelines, execução de etapas de treinamento, etc.

Certamente o algoritmo OneHotEncoder possui mais features e possibilidades de utilização, no entanto, para o trabalho atual, seguiremos com a simplicidade do get_dummies dado que: não há o foco no desenvolvimento do modelo de ML, as variáveis categóricas constituem um escopo asseguradamente fechado, os pontos positivos do OneHotEncoder não serão aproveitados e o custo computacional dele se demonstrou maior.

#### _get_dummies_

In [23]:
microdadosEnem = pd.get_dummies(microdadosEnem, columns= [
    'TP_SEXO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
    'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
    'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
    'Q023', 'Q024', 'Q025'
])

In [24]:
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,IN_TREINEIRO,CO_MUNICIPIO_PROVA,TP_LINGUA,MEDIA_NOTAS,...,Q022_E,Q023_A,Q023_B,Q024_A,Q024_B,Q024_C,Q024_D,Q024_E,Q025_A,Q025_B
3,10,1,1,1,1,1,0,1721000,1,475.34,...,False,True,False,False,True,False,False,False,False,True
4,7,1,1,1,1,1,0,3118601,0,639.36,...,False,False,True,False,True,False,False,False,False,True
5,13,2,2,1,1,1,0,5107602,0,649.62,...,True,True,False,False,True,False,False,False,False,True
6,7,1,3,1,1,1,0,2924009,1,534.98,...,False,True,False,True,False,False,False,False,True,False
9,2,1,1,1,2,2,0,3550308,0,495.04,...,False,False,True,False,True,False,False,False,False,True


In [25]:
if (microdadosEnem.dtypes == 'object').any():
    print("O DataFrame ainda possui colunas do tipo 'object'.")
else:
    print("O DataFrame não possui mais colunas do tipo 'object'.")

O DataFrame não possui mais colunas do tipo 'object'.


In [26]:
microdadosEnem.columns.values

array(['TP_FAIXA_ETARIA', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ESCOLA', 'IN_TREINEIRO',
       'CO_MUNICIPIO_PROVA', 'TP_LINGUA', 'MEDIA_NOTAS', 'TP_SEXO_F',
       'TP_SEXO_M', 'Q001_A', 'Q001_B', 'Q001_C', 'Q001_D', 'Q001_E',
       'Q001_F', 'Q001_G', 'Q001_H', 'Q002_A', 'Q002_B', 'Q002_C',
       'Q002_D', 'Q002_E', 'Q002_F', 'Q002_G', 'Q002_H', 'Q003_A',
       'Q003_B', 'Q003_C', 'Q003_D', 'Q003_E', 'Q003_F', 'Q004_A',
       'Q004_B', 'Q004_C', 'Q004_D', 'Q004_E', 'Q004_F', 'Q005_1',
       'Q005_2', 'Q005_3', 'Q005_4', 'Q005_5', 'Q005_6', 'Q005_7',
       'Q005_8', 'Q005_9', 'Q005_10', 'Q005_11', 'Q005_12', 'Q005_13',
       'Q005_14', 'Q005_15', 'Q005_16', 'Q005_17', 'Q005_18', 'Q005_19',
       'Q005_20', 'Q006_A', 'Q006_B', 'Q006_C', 'Q006_D', 'Q006_E',
       'Q006_F', 'Q006_G', 'Q006_H', 'Q006_I', 'Q006_J', 'Q006_K',
       'Q006_L', 'Q006_M', 'Q006_N', 'Q006_O', 'Q006_P', 'Q006_Q',
       'Q007_A', 'Q007_B', 'Q007_C', 'Q007_D

In [27]:
microdadosEnem.shape

(3701909, 156)

### Normalização dos dados

In [28]:
from sklearn.preprocessing import MinMaxScaler

aux_columns = microdadosEnem.columns
normalizador = MinMaxScaler().fit(microdadosEnem)

In [29]:
array_normalizado = normalizador.transform(microdadosEnem)

In [30]:
microdadosEnem = pd.DataFrame(array_normalizado)
microdadosEnem.columns = aux_columns

In [31]:
microdadosEnem.describe()

,TP_FAIXA_ETARIA,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,IN_TREINEIRO,CO_MUNICIPIO_PROVA,TP_LINGUA,MEDIA_NOTAS,...,Q022_E,Q023_A,Q023_B,Q024_A,Q024_B,Q024_C,Q024_D,Q024_E,Q025_A,Q025_B
count,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,...,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06,3.701909e+06
mean,2.139015e-01,2.641247e-01,4.219581e-01,2.576554e-01,2.074616e-01,1.923897e-01,1.425475e-01,4.713671e-01,5.036980e-01,6.142594e-01,...,1.976307e-01,7.482002e-01,2.517998e-01,4.382936e-01,4.421351e-01,8.336239e-02,2.557626e-02,1.063262e-02,2.112286e-01,7.887714e-01
std,1.993118e-01,9.919444e-02,2.039826e-01,5.315304e-02,2.442977e-01,2.952621e-01,3.496109e-01,2.354187e-01,4.999864e-01,9.832037e-02,...,3.982121e-01,4.340469e-01,4.340469e-01,4.961778e-01,4.966404e-01,2.764292e-01,1.578674e-01,1.025650e-01,4.081802e-01,4.081802e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.263158e-02,2.500000e-01,2.000000e-01,2.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,3.105134e-01,0.000000e+00,5.454033e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,1.578947e-01,2.500000e-01,4.000000e-01,2.500000e-01,0.000000e+00,0.000000e+00,0.000000e+00,4.806051e-01,1.000000e+00,6.053219e-01,...,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,3.157895e-01,2.500000e-01,6.000000e-01,2.500000e-01,3.333333e-01,5.000000e-01,0.000000e+00,5.830092e-01,1.000000e+00,6.778637e-01,...,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [32]:
microdadosEnem.to_csv(PATH_DATASET_TRATADO, index=False)